In [42]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [34]:
qb,wr,te,rb = clean_current('C:/Users/bobal/Downloads/DKSalaries (1).csv',9)

In [33]:
# Parse upcoming week file by position

def clean_current(csv, week):# Read in files

    df_upcoming = pd.read_csv(csv)
    def_2018 = pd.read_csv('input/def/fantasy-football-points-allowed-defense-by-position.csv')

    # Obtain home and away attributes
    
    df_upcoming['year']=2018
    df_upcoming['Week'] = week
    df_upcoming['home'] = df_upcoming['Game Info'].apply(lambda x: (x.split('@')[1]).split()[0])
    df_upcoming['away'] = df_upcoming['Game Info'].apply(lambda x: (x.split('@')[0]))
    opp = []
    for idx,team in enumerate(df_upcoming['TeamAbbrev']):
        if team == df_upcoming.iloc[idx]['home']:
            opp.append(df_upcoming.iloc[idx]['away'])
        else:
            opp.append(df_upcoming.iloc[idx]['home'])
    df_upcoming['Opponent'] = opp
    rb = []
    wr = []
    te = []
    qb= []
    for idx,pos in enumerate(df_upcoming['Position']):
        if pos == 'RB':
            rb.append(idx)
        elif pos =='WR':
            wr.append(idx)
        elif pos =='TE':
            te.append(idx)
        elif pos =='QB':
            qb.append(idx)
    df_upcoming.drop(['Name + ID', 'Position','Roster Position','Game Info','ID','AvgPointsPerGame','Salary'],axis=1, inplace=True)
    df_upcoming.rename(columns={'TeamAbbrev':'Team'}, inplace =True)

    # Sort current week dataframe

    qb_df = df_upcoming.iloc[qb]
    rb_df = df_upcoming.iloc[rb]
    wr_df = df_upcoming.iloc[wr]
    te_df = df_upcoming.iloc[te]
    
    return qb_df,wr_df,te_df, rb_df

In [84]:
te_cur = pd.read_csv('input/te/upcoming.csv')
te_cur.head()

,Name,Team,year,Week,home,away,Opponent
0,Travis Kelce,KC,2018,9,CLE,KC,CLE
1,Jordan Reed,WAS,2018,9,WAS,ATL,ATL
2,Greg Olsen,CAR,2018,9,CAR,TB,TB
3,David Njoku,CLE,2018,9,CLE,KC,KC
4,Trey Burton,CHI,2018,9,BUF,CHI,BUF


In [55]:
def_2018 = pd.read_csv('input/def/fantasy-football-points-allowed-defense-by-position.csv')

In [56]:
wk7 = pd.read_csv('input/def/wk7_def.csv')

In [57]:
position_dict = {'qb':'DraftKingsQuarterbackFantasyPointsAllowedAverage',
                    'wr':'DraftKingsWideReceiverFantasyPointsAllowedAverage',
                    'rb':'DraftKingsRunningbackFantasyPointsAllowedAverage',
                    'te':'DraftKingsTightEndFantasyPointsAllowedAverage'}

team_dict = {'New England Patriots':'NE',
            'Green Bay Packers':'GB',
            'Tampa Bay Buccaneers':'TB',
            'Kansas City Chiefs':'KC',
            'San Diego Chargers':'SD',
            'St. Louis Rams':'STL',
            'New Orleans Saints':'NO',
            'New York Jets':'NYJ',
            'New York Giants':'NYG',
            'San Francisco 49ers':'SF',
            'Los Angeles Rams':'LAR',
            'Jacksonville Jaguars':'JAX',
            'Los Angeles Chargers':'LAC'}


surface_dict={'KC':'grass',
             'CAR':'grass',
              'WAS':'grass',
              'CLE':'grass',
              'MIA':'grass',
              'PIT':'grass',
              'GB':'grass',
              'SF':'grass',
              'PHI':'grass',
              'LAR':'grass',
              'BAL': 'grass',
              'TEN':'grass',
              'OAK':'grass',
              'LAC':'grass',
              'TB':'grass',
              'CHI':'grass',
              'DEN':'grass',
              'ARI':'grass',
              'DAL':'turf',
              'SEA':'turf',
              'DET':'turf',
              'ATL':'turf',
              'NE':'turf',
              'IND':'turf',
              'NO':'turf',
              'NYG':'turf',
              'NYJ':'turf',
              'HOU':'turf',
              'BUF':'turf',
              'CIN':'turf',
              'MIN':'turf',
              'JAX':'grass'
              
             }

In [58]:
# Set wk8 defensive dataframe
def_2018.sort_values(position_dict['qb'], inplace=True)
def_2018.reset_index(inplace=True, drop=True)
def_2018['Rank'] = def_2018.index + 1
def_2018.index = def_2018['Team']

# set wk7 defensive dataframe
wk7.sort_values(position_dict['qb'], inplace=True)
wk7.reset_index(inplace=True, drop=True)
wk7['Rank'] = wk7.index + 1
wk7.index = wk7['Team']

In [85]:
dr = []
for idx, team in enumerate(te_cur['Opponent']):
    if team in list(def_2018.index):
        dr.append(def_2018.loc[team]['Rank'])
    else:
        dr.append(wk7.loc[team]['Rank'])

In [86]:
te_cur['def_rk'] = dr

In [87]:
te_cur.head()

,Name,Team,year,Week,home,away,Opponent,def_rk
0,Travis Kelce,KC,2018,9,CLE,KC,CLE,12
1,Jordan Reed,WAS,2018,9,WAS,ATL,ATL,23
2,Greg Olsen,CAR,2018,9,CAR,TB,TB,15
3,David Njoku,CLE,2018,9,CLE,KC,KC,10
4,Trey Burton,CHI,2018,9,BUF,CHI,BUF,9


In [88]:

sur = []
for team in te_cur['home']:
    sur.append(surface_dict[team])

In [89]:
te_cur['surface']=sur

In [90]:
home = []
for team in te_cur['Team']:
    if team in list(te_cur['home']):
        home.append('home')
    else:
        home.append('away')

In [91]:
te_cur['h/a'] = home

In [92]:
te_cur = one_hot(te_cur)

In [93]:
te_cur.head()

,Name,Team,year,Week,home,away,Opponent,def_rk,surface,h/a,h/a_encoded,surface_encoded,h/a_0,h/a_1,surface_0,surface_1
0,Travis Kelce,KC,2018,9,CLE,KC,CLE,12,grass,away,0,0,1.0,0.0,1.0,0.0
1,Jordan Reed,WAS,2018,9,WAS,ATL,ATL,23,grass,home,1,0,0.0,1.0,1.0,0.0
2,Greg Olsen,CAR,2018,9,CAR,TB,TB,15,grass,home,1,0,0.0,1.0,1.0,0.0
3,David Njoku,CLE,2018,9,CLE,KC,KC,10,grass,home,1,0,0.0,1.0,1.0,0.0
4,Trey Burton,CHI,2018,9,BUF,CHI,BUF,9,turf,away,0,1,1.0,0.0,0.0,1.0


In [94]:
te_cur.drop(['home','away','surface','h/a','surface_encoded','h/a_encoded','h/a_0','surface_1'], axis =1, inplace=True)

In [20]:
qb.reset_index(inplace=True, drop=True)

In [24]:
qb.drop(['Name + ID', 'Position','Roster Position','Game Info','ID','AvgPointsPerGame','Salary'],axis=1, inplace=True)

In [96]:
te_cur.to_csv('input/te/te_cur.csv', index=False)
wr_cur.to_csv('input/wr/wr_cur.csv', index=False)
rb_cur.to_csv('input/rb/rb_cur.csv', index=False)
qb_cur.to_csv('input/qb/qb_cur.csv', index=False)

In [39]:
qb.to_csv('input/qb/upcoming.csv', index=False)
rb.to_csv('input/rb/upcoming.csv', index=False)
wr.to_csv('input/wr/upcoming.csv',index=False)
te.to_csv('input/te/upcoming.csv',index=False)

In [ ]:
def add_defensive_rank(df, df1, position):
    df1.sort_values(position_dict[position], inplace=True)
    df1.reset_index(inplace=True, drop=True)
    df1['Rank']= df1.index+1
    df1.index=df1['Team']

# Map defensive rnking to eack player in the statist  ical dataframe 

    df['def_rk'] = list(map(lambda x: df1.loc[x]['Rank'], df['Opponent']))
    
    return df


In [38]:
# One hot encode all non numeric attributes


def one_hot(df):
    df.reset_index(inplace=True, drop=True)
    encoder = LabelEncoder()
    df['h/a_encoded'] = encoder.fit_transform(df['h/a'])
    df['surface_encoded'] = encoder.fit_transform(df['surface'])


    home =OneHotEncoder()
    surface = OneHotEncoder()

    X = home.fit_transform(df['h/a_encoded'].values.reshape(-1,1)).toarray()
    Xs = surface.fit_transform(df['surface_encoded'].values.reshape(-1,1)).toarray()

    dfOneHot = pd.DataFrame(X, columns = ["h/a_"+str(int(i)) for i in range(X.shape[1])])
    df = pd.concat([df, dfOneHot], axis=1)


    dfOneHot = pd.DataFrame(Xs, columns = ['surface_'+str(int(i)) for i in range(Xs.shape[1])])
    df = pd.concat([df, dfOneHot], axis=1)

    return df